In [1]:
import requests 
from bs4 import BeautifulSoup 
import re

In [2]:
jobs = []

def find_job(tech, location1):
    
    tech = tech.replace(' ', '-')
    location1 = location1.replace(' ', '-')
    
    for page in range(1,5):
       
        url = f'https://www.reed.co.uk/jobs/{tech}-jobs-in-{location1}?pageno={page}'
        response = requests.get(url)

        # print(response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')
        all_jobs = soup.find_all('article', class_='job-result-card')
      
        for job1 in all_jobs:

                title = job1.find('h2', class_='job-result-heading__title').text.strip()
            
            location = job1.find('li', class_='job-metadata__item job-metadata__item--location').text.split('\r\n')[1].strip()
          
            description = job1.find('p', class_='job-result-description__details').text.strip()

            salary = job1.find('li', class_='job-metadata__item job-metadata__item--salary').text.strip()

            posted_date, sep, tail = job1.find('div', class_='job-result-heading__posted-by').text.partition(' by')
            posted_date = posted_date.partition('\r\n                    ')[2]

            company_name = job1.find('a', class_='gtmJobListingPostedBy').text.strip()

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            job_link = job1.find('h2', class_='job-result-heading__title').find('a')
            responsibilities_link = 'https://www.reed.co.uk' + job_link.get('href')
        
            response2 = requests.get(responsibilities_link)
            soup2 = BeautifulSoup(response2.content, 'html.parser')
            
            page_text = soup2.get_text()

            
            tech_list = ['python', 'sql', 'java', 'c++', 'c#', 'spark', 'hadoop', 'azure', 
                       'crm', 'tableau', 'power bi', 'aws', 'ML', 'AI', 'scala', 'NLP', 
                       'HTML', 'Linux', 'nosql', 'cloud', 'Postgres ''Oracle','FastAPI']
            
            technology = [string for string in tech_list if string.lower() in page_text.lower()]

#             all_jobs = soup2.find_all('article', class_='job-result-card')
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            
            job = {
                    'title': title,
                    'company_name': company_name,
                    'posted_date': posted_date,
                    'location': location,
                    'salary': salary,
                    'description': description,
                    'technology': technology
                }
            
            jobs.append(job)
    return jobs



In [3]:
jobs = find_job(input("Enter technology: "), input("Enter location: "))

Enter technology: artificial intelligence
Enter location: united kingdom


In [4]:
import psycopg2

In [5]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="sufian3119",
    port = "5432"
)

cur = conn.cursor()

In [6]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS job2 (
        title TEXT NOT NULL,
        company_name TEXT NOT NULL,
        posted_date TEXT NOT NULL,
        location TEXT NOT NULL,
        salary TEXT NOT NULL,
        technology TEXT NOT NULL,
        description TEXT NOT NULL)
         
''')

In [7]:
from psycopg2.extras import execute_batch

In [8]:
query = "INSERT INTO job2 VALUES (%(title)s, %(company_name)s, %(posted_date)s, %(location)s,  %(salary)s, %(technology)s, %(description)s)"

execute_batch(cur, query, jobs)

conn.commit()
cur.close()
conn.close()